In [10]:
import sys
sys.path.insert(0, 'lib')

loader = DataLoader('yourdataset/*.npy',
                  'yourdataset/*.npy', labelNames)

from keras.layers import TimeDistributed
x = []
y = []
for step in range(30):
    trX, trY = loader.NextTrain()
    x.append(trX)
    y.append(trY)
    print(trX.shape,trX.mean(),trY)
x = np.array(x).reshape([-1,256,256,256])
y = np.array(y).reshape([-1])

Normal: 304
Abnormal: 600
Val all: 630 630
(1, 256, 256, 256, 1) 0.0906170941889286 [0]
(1, 256, 256, 256, 1) 0.10050584422424436 [1]
(1, 256, 256, 256, 1) 0.0834337396081537 [0]
(1, 256, 256, 256, 1) 0.08913979539647698 [1]
(1, 256, 256, 256, 1) 0.10763870715163648 [0]
(1, 256, 256, 256, 1) 0.09354960243217647 [1]
(1, 256, 256, 256, 1) 0.09614670975133777 [1]
(1, 256, 256, 256, 1) 0.0710067474283278 [0]
(1, 256, 256, 256, 1) 0.10795170697383583 [1]
(1, 256, 256, 256, 1) 0.08527430682443082 [1]
(1, 256, 256, 256, 1) 0.09115610551089048 [1]
(1, 256, 256, 256, 1) 0.09479357046075165 [0]
(1, 256, 256, 256, 1) 0.08650769479572773 [1]
(1, 256, 256, 256, 1) 0.10583823500201106 [1]
(1, 256, 256, 256, 1) 0.08447938854806125 [0]
(1, 256, 256, 256, 1) 0.0962220320943743 [0]
(1, 256, 256, 256, 1) 0.08822379354387522 [0]
(1, 256, 256, 256, 1) 0.08793994062580168 [1]
(1, 256, 256, 256, 1) 0.09873163094744086 [0]
(1, 256, 256, 256, 1) 0.1255127142649144 [1]
(1, 256, 256, 256, 1) 0.0990123595111072 [

In [13]:

from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed
from keras.models import Model, Sequential

# First, let's define a vision model using a Sequential model.
# This model will encode an image into a vector.
vision_model = Sequential()
vision_model.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=(256,256,1)))
#vision_model.add(Conv2D(8, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
#vision_model.add(Conv2D(16, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
#vision_model.add(Conv2D(16, (3, 3), activation='relu'))
#vision_model.add(Conv2D(16, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
#vision_model.add(Conv2D(16, (3, 3), activation='relu'))
#vision_model.add(Conv2D(16, (3, 3), activation='relu'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())
# This is our video encoded via the previously trained vision_model (weights are reused)
video_input = Input(shape=(256, 256, 256,1))
encoded_frame_sequence = TimeDistributed(vision_model)(video_input)  # the output will be a sequence of vectors
encoded_video = LSTM(256)(encoded_frame_sequence)  # the output will be a vector

output = Dense(2, activation='softmax')(encoded_video)
model = Model(inputs=video_input, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 256, 1)  0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 256, 4096)         5888      
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               4457472   
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 514       
Total params: 4,463,874
Trainable params: 4,463,874
Non-trainable params: 0
_________________________________________________________________


In [ ]:

print(x.shape, y.shape)
x = x.reshape([30,256,256,256,1])
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x, y, batch_size=1,epochs=1000)

(30, 256, 256, 256, 1) (30,)
Epoch 1/1000
30/30 [==============================] - 64s 2s/step - loss: 0.8227 - acc: 0.5667
Epoch 2/1000
30/30 [==============================] - 57s 2s/step - loss: 0.4754 - acc: 0.8667
Epoch 3/1000
30/30 [==============================] - 49s 2s/step - loss: 0.5255 - acc: 0.6667
Epoch 4/1000
25/30 [========================>.....] - ETA: 8s - loss: 0.5489 - acc: 0.6800 